# 영화 평점 분석 실습

실습관련 YouTube 영상  
https://youtu.be/krmthaX9WD4  
https://youtu.be/bbSDVNYUmb4  
https://youtu.be/JOska9sZVrw

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

## 영화 평점 데이터 적재 및 전처리

In [2]:
# 사용자 데이터 읽어오기
users = pd.read_csv('data/movielens/users.dat', 
                    sep = '::', 
                    engine = 'python', 
                    names = ['사용자아이디', '성별','연령','직업','지역'])
users.head()

,사용자아이디,성별,연령,직업,지역
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [3]:
# 평점 데이터 읽어오기
ratings = pd.read_csv('data/movielens/ratings.dat',
                      sep = '::', 
                      engine = 'python',
                      names = ['사용자아이디', '영화아이디','평점','타임스탬프'])
ratings.head()

,사용자아이디,영화아이디,평점,타임스탬프
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
# 영화데이터 읽어오기
movies = pd.read_csv('data/movielens/movies.dat',
                     sep = '::', 
                     engine = 'python',
                     names = ['영화아이디','영화제목','장르'], 
                     encoding = 'latin-1')
movies.head()

,영화아이디,영화제목,장르
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
#3개의 데이터프레임을 하나로 합치기
data = pd.merge(users,ratings,on='사용자아이디')
data.head()

,사용자아이디,성별,연령,직업,지역,영화아이디,평점,타임스탬프
0,1,F,1,10,48067,1193,5,978300760
1,1,F,1,10,48067,661,3,978302109
2,1,F,1,10,48067,914,3,978301968
3,1,F,1,10,48067,3408,4,978300275
4,1,F,1,10,48067,2355,5,978824291


In [6]:
data = pd.merge(data,movies,on='영화아이디')
data.head()

,사용자아이디,성별,연령,직업,지역,영화아이디,평점,타임스탬프,영화제목,장르
0,1,F,1,10,48067,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,M,56,16,70072,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,M,25,12,32793,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,M,25,7,22903,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,M,50,1,95350,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama


## 보고 싶은 영화  찾기 (평점 높은 영화 찾기)

In [7]:
data.영화제목.unique().size

3706

In [8]:
data.영화아이디.unique().size # 영화 제목이 안 겹친다는걸 알 수 있음

3706

In [9]:
# 영화들의 평점 평균을 구하여, 평점이 높은 영화 찾기 
data.pivot_table(index='영화제목', # 영화제목이 안 겹치니까
                 aggfunc='mean',
                 values='평점').sort_values(by='평점',ascending=False).head(10)

,평점
영화제목,
Ulysses (Ulisse) (1954),5.0
Lured (1947),5.0
Follow the Bitch (1998),5.0
Bittersweet Motel (2000),5.0
Song of Freedom (1936),5.0
One Little Indian (1973),5.0
Smashing Time (1967),5.0
Schlafes Bruder (Brother of Sleep) (1995),5.0
"Gate of Heavenly Peace, The (1995)",5.0


In [10]:
# 영화들의 평점 갯수도 함께 구하기
영화평점 = data.pivot_table(index='영화제목',aggfunc=['mean','count'],values=['평점'])
영화평점.columns = ['평균','갯수']

In [11]:
영화평점.head()

,평균,갯수
영화제목,,
"$1,000,000 Duck (1971)",3.027027,37
'Night Mother (1986),3.371429,70
'Til There Was You (1997),2.692308,52
"'burbs, The (1989)",2.910891,303
...And Justice for All (1979),3.713568,199


In [12]:
# 평점의 갯수가 2000개 이상이고 평균이 4.5이상인 영화를 평균으로 내림차순
영화평점[(영화평점.갯수>=2000) & (영화평점.평균 > 4.3)].sort_values(by='평균',ascending=False)

,평균,갯수
영화제목,,
"Shawshank Redemption, The (1994)",4.554558,2227
"Godfather, The (1972)",4.524966,2223
Schindler's List (1993),4.510417,2304
Raiders of the Lost Ark (1981),4.477725,2514
Star Wars: Episode IV - A New Hope (1977),4.453694,2991
"Sixth Sense, The (1999)",4.406263,2459
"Silence of the Lambs, The (1991)",4.351823,2578
Saving Private Ryan (1998),4.337354,2653
American Beauty (1999),4.317386,3428


## (실습) 여자들이 좋아하는 영화 찾기 (평점이 4.0 이상이고 여성 평점의 개수가 500개 이상인 영화)

In [13]:
ex1 = data[data.성별 == 'F']

In [14]:
ex1 = ex1.pivot_table(index='영화제목',values='평점',aggfunc=['mean','count'])

In [15]:
ex1.columns = ['여성평점평균','여성평점개수']

In [16]:
여성인기영화 = ex1[(ex1.여성평점평균>=4.0)&(ex1.여성평점개수>=500)]

In [17]:
# 다른 방법 - 성별로 영화평점 집계를 각각 수행
ex1_1 = data.pivot_table(index='영화제목',columns='성별',values='평점',aggfunc=['mean','count'])

In [18]:
ex1_1[(ex1_1[('mean','F')]>=4.0)&(ex1_1[('count','F')]>=500)]

mean            count  \
성별                                                         F         M      F   
영화제목                                                                            
American Beauty (1999)                              4.238901  4.347301  946.0   
Being John Malkovich (1999)                         4.159930  4.113636  569.0   
Braveheart (1995)                                   4.016484  4.297839  546.0   
Casablanca (1942)                                   4.300990  4.461340  505.0   
E.T. the Extra-Terrestrial (1982)                   4.089850  3.920264  601.0   
Fargo (1996)                                        4.217656  4.267780  657.0   
Forrest Gump (1994)                                 4.045031  4.105806  644.0   
L.A. Confidential (1997)                            4.106007  4.256678  566.0   
Matrix, The (1999)                                  4.128405  4.362235  514.0   
Princess Bride, The (1987)                          4.342767  4.288942  636.0   
Pulp Fiction (1994)                                 4.071956  4.346839  542.0   
Raiders of the Lost Ark (1981)                      4.332168  4.520597  572.0   
Saving Private Ryan (1998)                          4.114783  4.398941  575.0   
Schindler's List (1993)                             4.562602  4.491415  615.0   
Shakespeare in Love (1998)                          4.181704  4.099936  798.0   
Shawshank Redemption, The (1994)                    4.539075  4.560625  627.0   
Silence of the Lambs, The (1991)                    4.271955  4.381944  706.0   
Sixth Sense, The (1999)                             4.477410  4.379944  664.0   
Star Wars: Episode IV - A New Hope (1977)           4.302937  4.495307  647.0   
Star Wars: Episode V - The Empire Strikes Back ...  4.106481  4.344577  648.0   
Toy Story (1995)                                    4.187817  4.130552  591.0   
Wizard of Oz, The (1939)                            4.355030  4.203138  507.0   

                                                            
성별                                                       M  
영화제목                                                        
American Beauty (1999)                              2482.0  
Being John Malkovich (1999)                         1672.0  
Braveheart (1995)                                   1897.0  
Casablanca (1942)                                   1164.0  
E.T. the Extra-Terrestrial (1982)                   1668.0  
Fargo (1996)                                        1856.0  
Forrest Gump (1994)                                 1550.0  
L.A. Confidential (1997)                            1722.0  
Matrix, The (1999)                                  2076.0  
Princess Bride, The (1987)                          1682.0  
Pulp Fiction (1994)                                 1629.0  
Raiders of the Lost Ark (1981)                      1942.0  
Saving Private Ryan (1998)                          2078.0  
Schindler's List (1993)                             1689.0  
Shakespeare in Love (1998)                          1571.0  
Shawshank Redemption, The (1994)                    1600.0  
Silence of the Lambs, The (1991)                    1872.0  
Sixth Sense, The (1999)                             1795.0  
Star Wars: Episode IV - A New Hope (1977)           2344.0  
Star Wars: Episode V - The Empire Strikes Back ...  2342.0  
Toy Story (1995)                                    1486.0  
Wizard of Oz, The (1939)                            1211.0

### (심화) 여자들이 어떤 장르의 영화를 좋아하는지 찾아보자.(위에서 찾은 여성들이 좋아하는 영화들의 장르를 찾아낸 후, 어떤 장르가 많았는지 확인)

In [19]:
여성인기영화

,여성평점평균,여성평점개수
영화제목,,
American Beauty (1999),4.238901,946
Being John Malkovich (1999),4.159930,569
Braveheart (1995),4.016484,546
Casablanca (1942),4.300990,505
E.T. the Extra-Terrestrial (1982),4.089850,601
Fargo (1996),4.217656,657
Forrest Gump (1994),4.045031,644
L.A. Confidential (1997),4.106007,566
"Matrix, The (1999)",4.128405,514


In [20]:
movies.장르

0          Animation|Children's|Comedy
1         Adventure|Children's|Fantasy
2                       Comedy|Romance
3                         Comedy|Drama
4                               Comedy
5                Action|Crime|Thriller
6                       Comedy|Romance
7                 Adventure|Children's
8                               Action
9            Action|Adventure|Thriller
10                Comedy|Drama|Romance
11                       Comedy|Horror
12                Animation|Children's
13                               Drama
14            Action|Adventure|Romance
15                      Drama|Thriller
16                       Drama|Romance
17                            Thriller
18                              Comedy
19                              Action
20                 Action|Comedy|Drama
21                Crime|Drama|Thriller
22                            Thriller
23                        Drama|Sci-Fi
24                       Drama|Romance
25                       

In [21]:
movies2 = movies.set_index('영화제목')['장르']

In [22]:
ex2 = pd.concat([여성인기영화,movies2],axis = 1,join='inner')['장르']

In [23]:
ex2 = ex2.str.split('|',expand = True)

In [24]:
ex2[0].value_counts()

Action        7
Drama         4
Comedy        4
Crime         3
Thriller      1
Adventure     1
Animation     1
Children's    1
Name: 0, dtype: int64

In [25]:
ex2[0].value_counts().add(ex2[4].value_counts(),fill_value=0)

Action        7.0
Adventure     1.0
Animation     1.0
Children's    1.0
Comedy        4.0
Crime         3.0
Drama         4.0
Thriller      1.0
War           1.0
dtype: float64

In [26]:
여성인기장르 = Series()
for col in ex2.columns:
    여성인기장르 = 여성인기장르.add(ex2[col].value_counts(),fill_value=0)
여성인기장르

Action         7.0
Adventure      5.0
Animation      1.0
Children's     3.0
Comedy         6.0
Crime          3.0
Drama         12.0
Fantasy        2.0
Film-Noir      1.0
Musical        1.0
Mystery        1.0
Romance        4.0
Sci-Fi         4.0
Thriller       5.0
War            6.0
dtype: float64

## (실습) 남자와 여자의 호불호가 크게 갈리는 영화 10개를 출력 
### 전체 평점의 개수가 500개 이상인 영화만 대상으로 함.

In [27]:
ex4 = data.pivot_table(index='영화제목',columns='성별',aggfunc=['mean','count'],values='평점')

In [28]:
ex4[ex4[('count','F')]+ex4[('count','M')] >= 500]
# ex['count'].sum(axis=1)>=500

mean            count  \
성별                                                         F         M      F   
영화제목                                                                            
10 Things I Hate About You (1999)                   3.646552  3.311966  232.0   
101 Dalmatians (1961)                               3.791444  3.500000  187.0   
12 Angry Men (1957)                                 4.184397  4.328421  141.0   
13th Warrior, The (1999)                            3.112000  3.168000  125.0   
20,000 Leagues Under the Sea (1954)                 3.670103  3.709205   97.0   
2001: A Space Odyssey (1968)                        3.825581  4.129738  344.0   
28 Days (2000)                                      3.209424  2.977707  191.0   
Abyss, The (1989)                                   3.659236  3.689507  314.0   
Ace Ventura: Pet Detective (1994)                   3.000000  3.197917  190.0   
Addams Family, The (1991)                           3.186170  3.163498  188.0   
Adventures in Babysitting (1987)                    3.455782  3.208122  147.0   
Adventures of Buckaroo Bonzai Across the 8th Di...  3.308511  3.402321   94.0   
African Queen, The (1951)                           4.324232  4.223822  293.0   
Air Force One (1997)                                3.699588  3.555822  243.0   
Airplane II: The Sequel (1982)                      2.906250  2.995671   96.0   
Airplane! (1980)                                    3.656566  4.064419  396.0   
Aladdin (1992)                                      3.857143  3.756494  427.0   
Alice in Wonderland (1951)                          3.705882  3.692308  187.0   
Alien (1979)                                        3.888252  4.216119  349.0   
Alien Nation (1988)                                 3.433333  3.195946   60.0   
Alien: Resurrection (1997)                          2.708738  2.997041  103.0   
Aliens (1986)                                       3.802083  4.186684  288.0   
Alien³ (1992)                                       3.008264  3.042289  121.0   
Almost Famous (2000)                                4.220217  4.228731  277.0   
Amadeus (1984)                                      4.346734  4.213415  398.0   
American Beauty (1999)                              4.238901  4.347301  946.0   
American Graffiti (1973)                            3.940476  4.055556  252.0   
American History X (1998)                           4.181818  4.240741  154.0   
American Pie (1999)                                 3.539792  3.754545  289.0   
American President, The (1995)                      3.923483  3.718654  379.0   
...                                                      ...       ...    ...   
Wag the Dog (1997)                                  3.573222  3.459581  239.0   
Waking Ned Devine (1998)                            3.963964  3.839917  222.0   
War of the Worlds, The (1953)                       3.885246  3.855159   61.0   
Waterworld (1995)                                   2.593496  2.640152  123.0   
Wayne's World (1992)                                3.464286  3.646429  280.0   
Wedding Singer, The (1998)                          3.581081  3.445351  296.0   
Weird Science (1985)                                3.375000  3.435961  104.0   
West Side Story (1961)                              4.228374  3.953390  289.0   
Westworld (1973)                                    3.371795  3.428315   78.0   
What About Bob? (1991)                              3.429293  3.387722  198.0   
What's Eating Gilbert Grape (1993)                  3.879668  3.820796  241.0   
When Harry Met Sally... (1989)                      4.257028  3.987850  498.0   
While You Were Sleeping (1995)                      3.636066  3.385870  305.0   
White Men Can't Jump (1992)                         3.028777  3.231061  139.0   
Who Framed Roger Rabbit? (1988)                     3.569378  3.713251  418.0   
Whole Nine Yards, The (2000)                        3.296552  3.404814  145.0   
Wild T

In [31]:
# 3. 호불호가 크게 갈리는 영화 찾기
# 남자평점과 여자평점의 차이를 새로운 컬럼으로 추가
ex4['남녀차이'] = abs(ex4[('mean','F')] - ex4[('mean','M')])
ex4.head()

mean           count             남녀차이
성별                                    F         M     F      M          
영화제목                                                                    
$1,000,000 Duck (1971)         3.375000  2.761905  16.0   21.0  0.613095
'Night Mother (1986)           3.388889  3.352941  36.0   34.0  0.035948
'Til There Was You (1997)      2.675676  2.733333  37.0   15.0  0.057658
'burbs, The (1989)             2.793478  2.962085  92.0  211.0  0.168607
...And Justice for All (1979)  3.828571  3.689024  35.0  164.0  0.139547

In [33]:
# 3.2 호불호가 크게 갈리는 영화 10편 선택
ex4.sort_values(by='남녀차이',ascending=False).head(10)

mean           count  \
성별                                                         F         M     F   
영화제목                                                                           
Tigrero: A Film That Was Never Made (1994)          1.000000  4.333333   1.0   
Spiders, The (Die Spinnen, 1. Teil: Der Goldene...  4.000000  1.000000   2.0   
Neon Bible, The (1995)                              1.000000  4.000000   1.0   
James Dean Story, The (1957)                        4.000000  1.000000   2.0   
Country Life (1994)                                 5.000000  2.000000   1.0   
Enfer, L' (1994)                                    1.000000  3.750000   1.0   
Babyfever (1994)                                    3.666667  1.000000   3.0   
Stalingrad (1993)                                   1.000000  3.593750   1.0   
Woman of Paris, A (1923)                            5.000000  2.428571   1.0   
Cobra (1925)                                        4.000000  1.500000   1.0   

                                                              남녀차이  
성별                                                     M            
영화제목                                                                
Tigrero: A Film That Was Never Made (1994)           3.0  3.333333  
Spiders, The (Die Spinnen, 1. Teil: Der Goldene...   2.0  3.000000  
Neon Bible, The (1995)                               1.0  3.000000  
James Dean Story, The (1957)                         1.0  3.000000  
Country Life (1994)                                  2.0  3.000000  
Enfer, L' (1994)                                     4.0  2.750000  
Babyfever (1994)                                     1.0  2.666667  
Stalingrad (1993)                                   32.0  2.593750  
Woman of Paris, A (1923)                             7.0  2.571429  
Cobra (1925)                                         4.0  2.500000

## (실습) 연령대 별로 영화 평점 분석하기
### 연령대(10대 미만, 10대, 20대, ...50대) 컬럼을 추가한 후, 연령대로 집계를 수행하여 영화별 연령대별 영화평점 구하기

In [37]:
def generate_ages(x):
    if x< 10:
        return '10대 미만'
    elif x < 20:
        return '10대'
    elif x < 30:
        return '20대'
    elif x < 40:
        return '30대'
    elif x < 50:
        return '40대'
    elif x < 60:
        return '50대'
    else:
        return '60대 이상'

In [38]:
data.연령

0           1
1          56
2          25
3          25
4          50
5          18
6           1
7          25
8          25
9          45
10         18
11         25
12         45
13         18
14         25
15         18
16         25
17         50
18         25
19         50
20         35
21         56
22         25
23         45
24         56
25         45
26         25
27          1
28         35
29         25
           ..
1000179    25
1000180    56
1000181    18
1000182    25
1000183    45
1000184    45
1000185    25
1000186    18
1000187    50
1000188    35
1000189    25
1000190    25
1000191    25
1000192    18
1000193    18
1000194    25
1000195    56
1000196    25
1000197    56
1000198    56
1000199    56
1000200     1
1000201    35
1000202    35
1000203    45
1000204    18
1000205    35
1000206    18
1000207    18
1000208    25
Name: 연령, Length: 1000209, dtype: int64

In [40]:
data['연령대'] = data.연령.apply(generate_ages)
data.head()

,사용자아이디,성별,연령,직업,지역,영화아이디,평점,타임스탬프,영화제목,장르,연령대
0,1,F,1,10,48067,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,10대 미만
1,2,M,56,16,70072,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama,50대
2,12,M,25,12,32793,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama,20대
3,15,M,25,7,22903,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama,20대
4,17,M,50,1,95350,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama,50대


In [47]:
# np.digitize() 함수 활용
# 예시
sr = Series([15,1,23,31,18,42,60,80])
np.digitize(sr,[10,20,30,40,50])*10

array([10,  0, 20, 30, 10, 40, 50, 50])

In [52]:
data['연령대2'] = np.digitize(data.연령,[10,20,30,40,50])

In [53]:
data.연령대2 = data.연령대2.map({
    0 : '10대 미만',
    1 : '10대',
    2 : '20대',
    3 : '30대',
    4 : '40대',
    5 : '50대 이상'
})
data.연령대2

0          10대 미만
1          50대 이상
2             20대
3             20대
4          50대 이상
5             10대
6          10대 미만
7             20대
8             20대
9             40대
10            10대
11            20대
12            40대
13            10대
14            20대
15            10대
16            20대
17         50대 이상
18            20대
19         50대 이상
20            30대
21         50대 이상
22            20대
23            40대
24         50대 이상
25            40대
26            20대
27         10대 미만
28            30대
29            20대
            ...  
1000179       20대
1000180    50대 이상
1000181       10대
1000182       20대
1000183       40대
1000184       40대
1000185       20대
1000186       10대
1000187    50대 이상
1000188       30대
1000189       20대
1000190       20대
1000191       20대
1000192       10대
1000193       10대
1000194       20대
1000195    50대 이상
1000196       20대
1000197    50대 이상
1000198    50대 이상
1000199    50대 이상
1000200    10대 미만
1000201       30대
1000202       30대
1000203   

In [56]:
연령별평점 = data.pivot_table(index='영화제목',columns='연령대',aggfunc='mean',values='평점')
연령별평점[['10대 미만','10대','20대','30대','40대','50대']].head()

연령대,10대 미만,10대,20대,30대,40대,50대
영화제목,,,,,,
"$1,000,000 Duck (1971)",NaN,3.000000,3.090909,3.133333,2.000000,2.750000
'Night Mother (1986),2.0,4.666667,3.423077,2.904762,3.833333,3.750000
'Til There Was You (1997),3.5,2.500000,2.666667,2.900000,2.333333,2.600000
"'burbs, The (1989)",4.5,3.244444,2.652174,2.818182,2.545455,3.100000
...And Justice for All (1979),3.0,3.428571,3.724138,3.657143,4.100000,3.674419
